# Загрузка библиотек

In [269]:
import numpy as np
import pandas as pd
import typing

## Функция сигнала ошибки

In [270]:
def error_signal(desired, actual):
    return desired - actual

## Общая энергия ошибки

ej(n) = dj(n) - yj(n)

In [271]:
def error_energy(desired, actual):
    return 0.5 * np.sum(np.float_power(error_signal(desired, actual), 2))

## Функция стоимости - мера эффективности обучения

In [272]:
def cost_function(desired, actual):
    return error_energy(desired, actual)

## Сигмоида

In [273]:
def diff_sigmoid(S: float, a: float = 1.0):
    a * np.exp(-S * a) * sigmoid(S, a) ** 2

Функция сигмоиды:
![sigmoid](sigmoid.png)

In [274]:
def sigmoid(S, a: float = 1.0):
    return np.div(1.0, np.sum(np.exp(np.mul(S, -a)), 1))

## Сумматор

In [275]:
def summator(w, x):
    return np.dot(w, x)

## Выходной сигнал

In [276]:
def output_signal(x, w):
    return sigmoid(summator(x, w))

## Среднеквадратическая ошибка

def mse(error):
    np.sum(np.float_power(error, 2))

# Заполнение весов

Т.к. без понятия, как подобрать начальный вес, воспользуемся рандомом в границах от -2 до 2. Можно от 0 до 1

In [277]:
def weight_matrix_init(neuorons_count: int, vector_size: int):
    matrix = np.empty([neuorons_count, vector_size], dtype=float)
    return matrix - (matrix // 2) * 2

# Нормализация данных

In [278]:
def data_normalise(data: pd.Series):
    return (data - learn_saved_max) / (learn_saved_max - learn_saved_min)

# Денормализация данных

In [279]:
def data_denormalise(data: pd.Series):
    return data * (learn_saved_max - learn_saved_min) + learn_saved_min

# Выполнение слоя с сигмоидой

In [280]:
def runLayer(input_data, layer):
    return sigmoid(layer * input_data)


# Прямой проход

In [281]:
def forward(input_data, target):
    first_layer_res = runLayer(input_data, first_layer_weights)
    second_layer_res = runLayer(first_layer_res, second_layer_weights)
    return error_signal(target, second_layer_res), [first_layer_res, second_layer_res]

![title](local_grad_output.png)

In [282]:
def weight_output_delta(error_signal, layer_input):
    return error_signal * diff_sigmoid(layer_input)

![title](local_gard_hidden.png)

In [283]:
def weight_hidden_delta(layer_delta, layer_input, layer_weight):
    return diff_sigmoid(np.sum(layer_input)) * np.dot(layer_delta, layer_weight)

![title](delta_rule.png)

/\wji(n) = teta * delta(n) * yi(n)
teta = одна сотая, тысячная ... (на сколько мы строги в обучении ребёнка)
Есть два градиента: для выходного слоя, есть для скрытого слоя.

In [284]:
def delta_rule(delta, layer_output, tetta: float = 0.01):
    return tetta * pd.mul(delta, layer_output)

In [285]:
def back_propagation(
        error_sign,
        layers_inputs,
        first_layer_weights,
        second_layer_weights
):
    output_delta = weight_output_delta(error_sign, layers_inputs[1])
    second_layer_delta = weight_hidden_delta(output_delta, layers_inputs[0])
    second_layer_weights +=  layers_inputs[2] * output_delta
    first_layer_weights += layers_inputs[1] * second_layer_delta

# Подсчёт среднеквадратичной ошибки

In [286]:
# def mse():


# Регрессионная модель на примере со скважинами

### Произведем разбивку на тестовую и обучающую

In [287]:
df = pd.read_csv("Очищенный датасет скважины.csv")

## Перевод в numeric

In [288]:
for key in df.keys().values:
    column = df.get(key=key)

    if key != "дд.мм.гггг":
        column = pd.to_numeric(column, errors='coerce')
    else:
        column = pd.to_datetime(column, errors='coerce')

    df.pop(key)
    df[key] = column

In [289]:

target_set = ['КГФ т/тыс.м3', 'G_total кг/с']
learn = df.head(len(df) * 80 // 100)
test = df.tail(len(df) * 20 // 100)
print(len(df.columns))

22


### Нормализация данных

In [290]:
learn_saved_max = learn.max()
learn_saved_min = learn.min()

learn = data_normalise(learn)

### Инициализация весов слоёв

In [291]:
neurons_first_layer_count = 120
neurons_second_layer_count = 60

In [292]:
first_layer_weights = weight_matrix_init(neurons_first_layer_count, len(df.columns) - len(target_set))
second_layer_weights = weight_matrix_init(neurons_second_layer_count, neurons_first_layer_count)

# Обучение

In [293]:
epochs = 500
feature_data = learn.drop(target_set, axis=1)
target_data = learn[target_set]

In [294]:
for epoch in range(epochs):
    for row_id, _ in df.iterrows():
        error_sign, layers_inputs = forward(
            feature_data.iloc[row_id],
            target_data.iloc[row_id]
        )
        back_propagation(error_sign)

        if epoch % 100 == 0 and row_id == 1:
            print(f"${epoch} - ${error_sign}")


ValueError: operands could not be broadcast together with shapes (20,120) (20,) 

# Тест

Дифференцируем: dE(n) / dwij(n) = ej(n) * (-1) * phi'(vi(n)) * yj(n)
vi - сумма выходных сигналов нейрона

In [ ]:
# def dE_dw():
#     error_signal() * diff_sigmoid(summator(x, w)) * output_signal(x, w)

# Градиентный спуск

* Оригинальный - долго. Гоняет туда-сюда.
* Стохастический - за один прогон. То что запомнили за прямой прогон. Один раз туда, один раз прогнали и затем применили.
* Пакетный - берём не всю, а батчевый. Мы будем гнать не по 200, не по одной, а, допустим, по 10. И затем для пакета все это применим

Синопс, а не нейрон. Разные сущности.
Прогнали, запомнили. На какойто посчитали для входного, затем выходного. Применили веса.

# Регуляризация (по желанию)

Рекомендутся сделать элементарный стохастический град спуск. Считать по формулам со слайда

Сигмоида только с положительными. Гипер Тангенс - знак. ReLu про другое.
Главное, что число гиперпараметров сети должно было хватить, чтобы обобщить.
Нейронка не должна быть маленькой.
Два слоя по 50 нейронов. (Чувак показывал, всё получилось)
R2 score посчитать, MSE. Обобщаем на многомерный случай. ROC-AUC можно.

# 70% - 30%. Обучать на тестовой